In [1]:
from scipy.optimize import linprog

In [2]:
obj = [-1, -2]
lhs_ineq = [[ 2,  1],  # Red constraint left side
[-4,  5],  # Blue constraint left side
[ 1, -2]]  # Yellow constraint left side

rhs_ineq = [20,  # Red constraint right side
10,  # Blue constraint right side
2]  # Yellow constraint right side

lhs_eq = [[-1, 5]]  # Green constraint left side
rhs_eq = [15]       # Green constraint right side

In [3]:
bnd = [(0, float("inf")),  # Bounds of x
(0, float("inf"))]  # Bounds of y

In [4]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
...               A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
...               method="revised simplex")
opt

     con: array([1.77635684e-15])
     fun: -16.818181818181817
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.        , 18.18181818,  3.36363636])
  status: 0
 success: True
       x: array([7.72727273, 4.54545455])

In [5]:
from pulp import * # pulp needs to be installed into the kernel first
import pandas as pd
import numpy as np

n_warehouses = 2
n_customers = 4

# Cost Matrix
cost_matrix = np.array([[1, 3, 0.5, 4],
                       [2.5, 5, 1.5, 2.5]])
# Demand Matrix
cust_demands = np.array([35000, 22000, 18000, 30000])

# Supply Matrix
warehouse_supply = np.array([60000, 80000])

model = LpProblem("Supply-Demand-Problem", LpMinimize)

variable_names = [str(i)+str(j) for j in range(1, n_customers+1) for i in range(1, n_warehouses+1)]
variable_names.sort()
print("Variable Indices:", variable_names)

DV_variables = LpVariable.matrix("X", variable_names, cat = "Integer", lowBound= 0 )
allocation = np.array(DV_variables).reshape(2,4)
print("Decision Variable/Allocation Matrix: ")
print(allocation)

obj_func = lpSum(allocation*cost_matrix)
print(obj_func)
model +=  obj_func
print(model)

Variable Indices: ['11', '12', '13', '14', '21', '22', '23', '24']
Decision Variable/Allocation Matrix: 
[[X_11 X_12 X_13 X_14]
 [X_21 X_22 X_23 X_24]]
X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24
Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer



In [6]:
#Supply Constraints
for i in range(n_warehouses):
    print(lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i])
    model += lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i] , "Supply Constraints " + str(i)

X_11 + X_12 + X_13 + X_14 <= 60000
X_21 + X_22 + X_23 + X_24 <= 80000


In [7]:
# Demand Constraints
for j in range(n_customers):
    print(lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
    model += lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j] , "Demand Constraints " + str(j)

X_11 + X_21 >= 35000
X_12 + X_22 >= 22000
X_13 + X_23 >= 18000
X_14 + X_24 >= 30000


In [8]:
model.writeMPS("Supply_demand_prob.lp") # write problem set-up to a text file

[X_11, X_12, X_13, X_14, X_21, X_22, X_23, X_24]

In [9]:
from pulp import GLPK # A working solver needs to be loaded explicit on ARM!

#model.solve()

model.solve(solver=GLPK(msg=True))

status =  LpStatus[model.status]

print(status)

Optimal


In [10]:
print("Total Cost:", model.objective.value())

# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total Cost: 200000.0
X_11 = 35000
X_12 = 22000
X_13 = 3000
X_14 = 0
X_21 = 0
X_22 = 0
X_23 = 15000
X_24 = 30000


In [11]:
# Warehouse 1 and Warehouse 2 required capacity

for i in range(n_warehouses):
    print("Warehouse ", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_customers)))

Warehouse  1
60000
Warehouse  2
45000
